In [1]:
from mmnl import MMNL
import numpy as np
import pickle
import pandas as pd


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authorize Google Drive to upload files to the drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
def load_data(path):
    dat = np.load(path)
    X = dat[:, :-1]
    Y = np.reshape(dat[:, -1], (-1, 1))
    return X, Y
X, Y = load_data('data/data.npy')

Load data

In [3]:
%%time
infile = open('./data/500_MC_dgp_uts.p', 'rb')
big_dict = pickle.load(infile)
Y_dgp = big_dict['theta: [ 1.5  1.  -1.1  0.4  0.1  0.6]']
print(Y_dgp.shape)

(2798, 500)
Wall time: 14 ms


In [4]:
%%time
result = []
draws = 25
start = 0
end = 50
method = 'QMC'
for i in range(start,end):
    print('iteration: %i' %(i+1))
    qmc = MMNL(X,Y_dgp[:,i],draws,3,method=method)
    solution = qmc.solver()
    result.append(solution)
    if i%10==9:
        pickle.dump( result, open( "./resultaten/%s(%i)/%i-%i_%s(%i)_utsgdp_results.p" %(method,draws,i-9,i,method,draws), "wb" ) )
        result = []
#     f = drive.CreateFile()
#     f.SetContentFile("./resultaten/%i-%i_%s(%i)_results.p" %(i,i+10,method,draws))
#     f.Upload()

iteration: 1
 final_simplex: (array([[ 0.98136541, -0.17191631, -1.68246229, -0.08905527,  1.54752668,
        -0.14682326],
       [ 0.98135974, -0.1719141 , -1.68245506, -0.08909279,  1.54752234,
        -0.14683493],
       [ 0.98136479, -0.17192077, -1.68244964, -0.08901872,  1.54753371,
        -0.14683073],
       [ 0.98136118, -0.17191163, -1.68246473, -0.08903505,  1.54754555,
        -0.1468569 ],
       [ 0.98136414, -0.17191869, -1.6824541 , -0.08902804,  1.5475553 ,
        -0.14680482],
       [ 0.98136445, -0.17191533, -1.68246289, -0.0889845 ,  1.54753066,
        -0.14683642],
       [ 0.98138596, -0.17193339, -1.68246362, -0.08906063,  1.54758129,
        -0.14685823]]), array([2458.20709421, 2458.20709422, 2458.20709422, 2458.20709423,
       2458.20709424, 2458.20709424, 2458.20709425]))
           fun: 2458.2070942112837
       message: 'Optimization terminated successfully.'
          nfev: 848
           nit: 539
        status: 0
       success: True
            

KeyboardInterrupt: 

In [11]:
pickle.dump( result, open( "./resultaten/BQMC(8)/90_BQMC(8)_results.p", "wb" ) )

In [10]:
x_est = np.array(result)
x_est.shape

(100, 6)

In [11]:
np.mean(x_est, axis=0)

array([ 4.58777441,  2.65487859, -7.12572568,  7.45172249,  2.22983804,
        5.23629935])